In [81]:
import random as ra
import numpy as np
import math

In [96]:
# Cards will be determined by the following numbers: 3 = 3, 4 = 4... J = 11, Q = 12, K = 13, A = 14, 2 = 15
# Suits will be determined as follows: +0 => Spades, +0.25 => Clubs, +0.5 => Diamonds, +0.75 => Hearts

fulldeck = [x*0.25 for x in range(12, 64)]
ra.shuffle(fulldeck)

def reset_deck():
    global fulldeck
    fulldeck = [x*0.25 for x in range(12, 64)]
    ra.shuffle(fulldeck)

In [83]:
# Returns the unicode number in decimal int of a card
def card_to_picture(card):
    if (card >= 14):
        card = card-13
    value = math.floor(card)
    #print(value)
    suit = 10+4*(card%1.0)
    if suit == 11:
        suit = 13
    elif suit == 13:
        suit = 11
    #print(suit)
    
    #king is actually E, queen is D
    if value == 13 or value == 12:
        value += 1
    return int(int("1f000",16)+16*suit+value)
    
#print(chr(card_to_picture(12.25)))

In [84]:
# Returns the unicode representation of the hand
def show_hand(hand0):
    hand = ""
    for i in hand0:
        hand += chr(card_to_picture(i))
    return hand

In [93]:
# Deal hands
p1 = fulldeck.copy()[0:13]
p2 = fulldeck.copy()[13:26]
p3 = fulldeck.copy()[26:39]
p4 = fulldeck.copy()[39:52]

p1.sort()
p2.sort()
p3.sort()
p4.sort()

def deal_hands():
    reset_deck()
    global p1, p2, p3, p4
    p1 = fulldeck.copy()[0:13]
    p2 = fulldeck.copy()[13:26]
    p3 = fulldeck.copy()[26:39]
    p4 = fulldeck.copy()[39:52]
    p1.sort()
    p2.sort()
    p3.sort()
    p4.sort()

In [86]:
# Rules are as follows: single = 0, pair = 1, triple = 2, three-straight = 3, four-straight = 4, five-straight = 5, 6 = new rule

# Also gonna initialize topcard and last played move and turn

rule = 6
lastplay = [2.0]
topcard = 2.0
turn = 1

In [87]:
# Function that gets valid moves given a hand, rule, card-to-beat
def get_moves(hand0, topcard=topcard):
    hand = hand0.copy()
    hand.sort()
    
    #remove zeros
    for i in range(len(hand)-1,-1,-1):
        if hand[i] == 0:
            hand.pop(i)
    
    values = hand.copy()
    for i in range(len(values)):
        values[i] = math.floor(values[i])
    #print(values)
    suits = hand.copy()
    for i in range(len(suits)):
        suits[i] = suits[i]%1.0
    #print(suits)

    singles = []
    # get singles
    for i in range(len(hand)):
        if hand[i] != 0:
            singles += [[hand[i]]]
            
    # get pairs
    pairs = []
    for i in range(len(values)):
        for j in range(len(values)):
            if values[i] == values[j] and i != j:
                if [hand[i], hand[j]] in pairs:
                    pairs.remove([hand[i], hand[j]])
                pairs += [[hand[i], hand[j]]]
    for i in pairs:
        for j in pairs:
            if sorted(i) == sorted(j) and i!=j:
                pairs.remove(j)
    #print(pairs)
                
    # get triples
    triples = []
    for i in range(len(values)):
        for j in range(len(values)):
            for k in range(len(values)):
                if values[i] == values[j] == values[k] and i != j !=k and i!=k:
                    triples += [[hand[i], hand[j], hand[k]]]
    for i in triples:
        for j in triples:
            if sorted(i) == sorted(j) and i!=j: 
                triples.remove(j)
    #print(triples)
    
    # get 3-straights
    three_straights = []
    for i in range(len(values)):
        for j in range(len(values)):
            for k in range(len(values)):
                if values[k] == values[j]+1 and values[j] == values[i]+1:
                    three_straights += [[hand[i], hand[j], hand[k]]]
                    
    # get 4-straights
    four_straights = []
    for i in range(len(values)):
        for j in range(len(values)):
            for k in range(len(values)):
                for l in range(len(values)):
                    if values[k] == values[j]+1 and values[j] == values[i]+1 and values[l] == values[k]+1:
                        four_straights += [[hand[i], hand[j], hand[k], hand[l]]]
                        
    # get 5-straights
    five_straights = []
    for i in range(len(values)):
        for j in range(len(values)):
            for k in range(len(values)):
                for l in range(len(values)):
                    for m in range(len(values)):
                        if values[k] == values[j]+1 and values[j] == values[i]+1 and values[l] == values[k]+1 and values[m] == values[l]+1:
                            five_straights += [[hand[i], hand[j], hand[k], hand[l], hand[m]]]
    
    # add proper moves based on rule
    moves = []
    if rule == 0 or rule == 6:
        moves += singles
    elif rule == 1 or rule == 6:
        moves += pairs
    elif rule == 2 or rule == 6:
        moves += triples
    elif rule == 3 or rule == 6:
        moves += three_straights
    elif rule == 4 or rule == 6:
        moves += four_straights
    elif rule == 5 or rule == 6:
        moves += five_straights
    
    #print(moves)
    # remove moves below topcard
    valid_moves = []
    for i in moves:
        if max(i) > float(topcard):
            valid_moves += [i]
    
    return valid_moves

In [88]:
# Function that translates moves to tiers
# Returns a 13-vector with 1 if that index (tier) is playable 
def tier_moves(hand, valid_moves):
    tierlist = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for i in valid_moves:
        tierlist[hand.index(max(i))] = 1
    return tierlist
        
#print(tier_moves(p1, get_moves(p1, 10)))
#print(p1)

# Function that translates a tier to a move
def tier_to_move(tier, hand=p4):
    movelist = get_moves(hand)
    for move in movelist:
        if hand[tier] in move:
            return move

In [89]:
# Function that plays a move
def play(player, move):
    movelength = len(move)
    for i in range(movelength):
        player.remove(move[i])
    
    global topcard
    topcard = max(move)
    global lastplay
    lastplay = move
    
    #check what move it is
    move.sort()
    values = move.copy()
    for i in range(len(values)):
        values[i] = math.floor(values[i])
    suits = move.copy()
    for i in range(len(suits)):
        suits[i] = suits[i]%1.0
    
    global rule
    if movelength == 1:
        rule = 0
    elif movelength == 2:
        rule = 1
    elif movelength == 3:
        if values[0] == values[1]:
            rule = 2
        elif values[1] == values[0]+1:
            rule = 3
    elif movelength == 4:
        rule = 4
    elif movelength == 5:
        rule = 5
    
    print("New rule is:", rule)
    print("New topcard is:", topcard)

In [112]:
def newgame():
    deal_hands()
    global rule
    rule = 0
    global topcard
    topcard = 5.0
    app.refresh()

In [113]:
import tkinter as tk

class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.grid()
        self.create_widgets()

    def create_widgets(self):
        #Get the unicode strings to render
        p1hand = ""
        for i in p1:
            p1hand += chr(card_to_picture(i))

        lastplayed = ""
        for i in lastplay:
            lastplayed += chr(card_to_picture(i))
            
        movelist = get_moves(p1)
        
        #these are the widgets
        self.hi_there = tk.Button(self)
        self.hi_there["text"] = "Reset"
        self.hi_there["command"] = lambda: newgame()
        self.hi_there.grid(row = 0)
        
        # Last play
        self.lp = tk.Label(self, text=lastplayed, font = ("Times", 120))
        self.lp.grid(row = 1, columnspan = 6)
        
        # Dashboard
        self.dashboard = tk.Label(self, text="P2: "+str(len(p2))+" cards left, "+"P3: "+str(len(p3))+" cards left, "+"P4: "+str(len(p4))+" cards left", font = ("Courier", 10))
        self.dashboard.grid(row = 2, columnspan = 5)
        
        # My Hand
        self.hand = tk.Label(self, text=p1hand, font = ("Times", 120))
        self.hand.grid(row = 3, columnspan = 6)

        self.move = []
        for i in movelist:
            self.move.append(tk.Button(self, text=str(i), font = ("Courier", 10),
                              command=self.button_moves(i)))
        for j in range(len(movelist)):
            self.move[j].grid(row = 4+math.floor(j/5), column = j%5)
            
    def button_moves(self, move):
        play(p1, move)
        self.refresh()

    def refresh(self):
        self.hand.destroy()
        self.lp.destroy()
        self.dashboard.destroy()
        for i in self.move:
            i.destroy()
        self.create_widgets()
        
rule = 0

root = tk.Tk()
app = Application(master=root)
app.mainloop()

New rule is: 0
New topcard is: 3.0
New rule is: 0
New topcard is: 3.75
New rule is: 0
New topcard is: 5.25
New rule is: 0
New topcard is: 5.75
New rule is: 0
New topcard is: 6.0
New rule is: 0
New topcard is: 8.0
New rule is: 0
New topcard is: 9.0
New rule is: 0
New topcard is: 9.75
New rule is: 0
New topcard is: 10.75
New rule is: 0
New topcard is: 11.0
New rule is: 0
New topcard is: 12.25
New rule is: 0
New topcard is: 14.0
New rule is: 0
New topcard is: 14.25


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-113-a062f72944d2>", line 25, in <lambda>
    self.hi_there["command"] = lambda: newgame()
  File "<ipython-input-112-0b6d96908552>", line 7, in newgame
    app.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    sel

New rule is: 0
New topcard is: 3.0
New rule is: 0
New topcard is: 3.25
New rule is: 0
New topcard is: 4.25
New rule is: 0
New topcard is: 6.0
New rule is: 0
New topcard is: 9.0
New rule is: 0
New topcard is: 9.25
New rule is: 0
New topcard is: 9.5
New rule is: 0
New topcard is: 12.25
New rule is: 0
New topcard is: 12.5
New rule is: 0
New topcard is: 13.25
New rule is: 0
New topcard is: 13.5
New rule is: 0
New topcard is: 14.0
New rule is: 0
New topcard is: 15.0


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-113-a062f72944d2>", line 25, in <lambda>
    self.hi_there["command"] = lambda: newgame()
  File "<ipython-input-112-0b6d96908552>", line 7, in newgame
    app.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    sel

New rule is: 0
New topcard is: 3.75
New rule is: 0
New topcard is: 4.0
New rule is: 0
New topcard is: 4.25
New rule is: 0
New topcard is: 5.0
New rule is: 0
New topcard is: 5.75
New rule is: 0
New topcard is: 8.5
New rule is: 0
New topcard is: 11.25
New rule is: 0
New topcard is: 11.5
New rule is: 0
New topcard is: 11.75
New rule is: 0
New topcard is: 13.25
New rule is: 0
New topcard is: 14.0
New rule is: 0
New topcard is: 15.0
New rule is: 0
New topcard is: 15.75


...

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-113-a062f72944d2>", line 25, in <lambda>
    self.hi_there["command"] = lambda: newgame()
  File "<ipython-input-112-0b6d96908552>", line 7, in newgame
    app.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    sel

New rule is: 0
New topcard is: 3.25
New rule is: 0
New topcard is: 3.75
New rule is: 0
New topcard is: 4.0
New rule is: 0
New topcard is: 4.75
New rule is: 0
New topcard is: 8.0
New rule is: 0
New topcard is: 9.25
New rule is: 0
New topcard is: 11.0
New rule is: 0
New topcard is: 11.75
New rule is: 0
New topcard is: 12.25
New rule is: 0
New topcard is: 13.5
New rule is: 0
New topcard is: 14.0
New rule is: 0
New topcard is: 15.5
New rule is: 0
New topcard is: 15.75


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-113-a062f72944d2>", line 25, in <lambda>
    self.hi_there["command"] = lambda: newgame()
  File "<ipython-input-112-0b6d96908552>", line 7, in newgame
    app.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    sel

New rule is: 0
New topcard is: 5.0
New rule is: 0
New topcard is: 5.25
New rule is: 0
New topcard is: 6.75
New rule is: 0
New topcard is: 7.25
New rule is: 0
New topcard is: 8.75
New rule is: 0
New topcard is: 9.5
New rule is: 0
New topcard is: 10.75
New rule is: 0
New topcard is: 11.0
New rule is: 0
New topcard is: 12.25
New rule is: 0
New topcard is: 13.75
New rule is: 0
New topcard is: 14.0
New rule is: 0
New topcard is: 15.25
New rule is: 0
New topcard is: 15.5


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-113-a062f72944d2>", line 25, in <lambda>
    self.hi_there["command"] = lambda: newgame()
  File "<ipython-input-112-0b6d96908552>", line 7, in newgame
    app.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    self.create_widgets()
  File "<ipython-input-113-a062f72944d2>", line 43, in create_widgets
    command=self.button_moves(i)))
  File "<ipython-input-113-a062f72944d2>", line 49, in button_moves
    self.refresh()
  File "<ipython-input-113-a062f72944d2>", line 57, in refresh
    sel

In [31]:
import sys
from contextlib import closing
from io import StringIO
from gym import utils
from gym.envs.toy_text import discrete

class KillerEnv(discrete.DiscreteEnv):
    def __init__(self):
        self.action_space = spaces.Discrete(14)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(14),
            spaces.Discrete(13)))

        # Start the first game
        self.reset()

    #Types of Actions: 1-13 => play a move from tier 1-13, 0 => pass,  
    def step(self, action):
        #this is for the machine
        assert self.action_space.contains(action)
        if action == 0:
            reward = 0
        else:
            if tier_moves(p4, get_moves(p4))[action-1] == 0:
                reward = -100
            else:
                myMove = tier_to_move(action-1)
                play(p4, myMove)
                reward = len(myMove)
        if len(p1) == 0 or len(p2) == 0 or len(p3) == 0 or len(p4) == 0:
            done = True
        else:
            done = False
            
        #this is for user input
        open_app()
        
        return self._get_obs(), reward, done, {}

    def _get_obs(self):
        return (len(p1), tier_moves(p4, get_moves(p4)))

    def reset(self):
        deal_hands()
        global rule
        rule = 0
        global topcard
        topcard = 2.0
        global turn
        if 3.0 in p1:
            turn = 1
        elif 3.0 in p2:
            turn = 2
        elif 3.0 in p3:
            turn = 3
        else:
            turn = 4
        